# Imports

In [1]:
import dft_toolbox as dft
import numpy as np
import scipy.stats as st
import glob
import matplotlib.pyplot as plt

# Create G16 Input Files

Variables

In [2]:
nodes = 5
mem = 91
partition = "mpiqdr"

GasRouteSection = "rb3lyp/aug-cc-pvdz empiricaldispersion=gd3 int(grid=ultrafine, acc2e=11) scf=(tight,maxcycles=500) iop(1/152=1000) opt=(maxcycles=1000,cartesian) freq=noraman nosymm"
# opt=(maxstep=1) iop(1/8=1)
PCMRouteSection = "geom=check rb3lyp-aug-cc-pvdz empiricaldispersion=gd3 int(grid=ultrafine, acc2e=11) scf=(tight,maxcycles=500) scrf=(iefpcm,solvent=water,externaliteration,1stvac,read) nosymm"

Loop through files

In [3]:
coordFiles = glob.glob("Ex01_supporting_files/*.xyz")
for num, coordFile in enumerate(coordFiles):
  dft.create_g16_input(
    fname=f"sim{num:03}",
    GasRouteSection=GasRouteSection,
    PCMRouteSection=PCMRouteSection,
    coordFile=coordFile
  )
  dft.create_slurm_script(
    fname=f"sim{num:03}",
    filename_g16=f"sim{num:03}",
    nodes=nodes,
    mem=mem,
    partition=partition
  )

# Create Arkane Input Files

Loop through files

In [4]:
freqOutput = glob.glob("Ex01_supporting_files/*_gas.log")
PCMOutput = glob.glob("Ex01_supporting_files/*_PCM.log")

for i in range(len(freqOutput)):
    dft.create_arkane_input(
      f"sim{i:03}",
      freq_log=freqOutput[i],
      #pcm_log=PCMOutput[i],
      linear=False,
      spinMultiplicity=1,
      opticalIsomers=1
    )

# pQCT Example with 50 Completed Clusters

Preliminary variables

In [5]:
n_water = [5 for i in range(50)] # shape (N,)
temperatures = np.linspace(10, 50, num=20) + 273.15 # shape (20,)

### Gas Phase Thermo Calculations
Loop through temperatures

In [6]:
# shape (20,)
G_water = []
H_water = []
S_water = []

# shape (20,)
G_Na = []
H_Na = []
S_Na = []

# shape (20,N)
G_clusters = []
H_clusters = []
S_clusters = []

In [7]:
for temp in temperatures:
  # water
  values = dft.calc_thermo_Arkane("Ex01_supporting_files/refChem.inp", temperature=temp)[1] # H2O
  G_water.append(values[3])
  H_water.append(values[1])
  S_water.append(values[2])
  
  # ion
  values = dft.calc_thermo_Arkane("Ex01_supporting_files/refChem.inp", temperature=temp)[2] # Na+
  G_Na.append(values[3])
  H_Na.append(values[1])
  S_Na.append(values[2])
  
  # cluster
  values = dft.calc_thermo_Arkane("Ex01_supporting_files/clustersChem.inp", temperature=temp) # shape (N,4)
  values = np.transpose(values) # shape (4,N)
  G_clusters.append(values[3])
  H_clusters.append(values[1])
  S_clusters.append(values[2])

In [8]:
G_water = np.array(G_water)
H_water = np.array(H_water)
S_water = np.array(S_water)    

G_Na = np.array(G_Na)
H_Na = np.array(H_Na)
S_Na = np.array(S_Na)

print(G_clusters[0])

[-247.9360456  -245.35682012 -247.94293733 -245.40245133 -245.39746701
 -245.3993559  -245.40114083 -248.5359254  -245.39933083 -245.62384138
 -245.39856107 -245.39624916 -247.9446442  -245.36697877 -245.39986564
 -245.40141652 -245.40147799 -244.82428667 -245.36061522 -245.39664481
 -245.39058053 -244.97907855 -247.95611283 -245.55596341 -245.39588895
 -247.93404004 -247.98448531 -245.41844669 -245.40308681 -245.38827872
 -245.38884719 -245.39419352 -245.39589954 -245.38688901 -248.91066469
 -245.3959155  -245.40220325 -245.39492827 -245.39271056 -247.94479179
 -249.55863413 -245.3770367  -247.93860292 -245.38580035 -247.9257134
 -245.39292924 -245.38843924 -245.39761763 -245.39491391 -245.3599062 ]


### Extract PCM solvation energies

In [9]:
cluster_PCM_dG = [dft.dGSolvPCM(fname) for fname in PCMOutput]
H2O_PCM_dG = dft.dGSolvPCM("Ex01_supporting_files/H2O_pcm_gsolv.log")
print(cluster_PCM_dG)

[-32.67, -35.37, -32.67, -35.36, -35.36, -35.36, -35.36, -32.66, -35.36, -38.58]


Free energy **in solution** for first 10 samples

In [10]:
G_aq_283K = [dft.calc_pQCT(G_clusters[0][i], G_water[0], cluster_PCM_dG[i], n_water[i], dG_solv_H2O=H2O_PCM_dG, temp=temperatures[0]) for i in range(10)]
print(G_aq_283K)

[42.59834375648746, 42.47756922722124, 42.59145202191562, 42.44193802641917, 42.44692234379233, 42.44503345292612, 42.443248518545865, 42.008463950441175, 42.44505852151213, 39.000547975810925]


Free energy **of solvation**

In [11]:
R = 0.0019872042586408316  # kcal/(mol*K)

In [12]:
dG_solv_283K = G_aq_283K - G_Na[0] - (R * temperatures[0] * np.log(24.46)) # shape (N,)
print(dG_solv_283K)

[ -99.31113022  -99.43190475  -99.31802196  -99.46753595  -99.46255163
  -99.46444052  -99.46622546  -99.90101003  -99.46441546 -102.908926  ]


Boltzmann Averaging and Bootstrapping

In [13]:
beta = [1/(R*temperatures[0]) for i in range(len(dG_solv_283K))]

mid = dft.boltzmannG(dG_solv_283K,beta)
bootstrappedCI = st.bootstrap((dG_solv_283K,beta),dft.boltzmannG,paired=True,vectorized=False,n_resamples=10000)
lowerBar = bootstrappedCI.confidence_interval[1]-mid
upperBar = mid-bootstrappedCI.confidence_interval[0]

print(mid)
print(lowerBar)
print(upperBar)

AttributeError: module 'numpy' has no attribute 'float128'